In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.core.display import display, HTML

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import plotly.express as px
#import plotly.offline as pyo
import folium
import plotly.graph_objects as go
import seaborn as sns
import ipywidgets as widgets

ModuleNotFoundError: No module named 'plotly.express'

In [ ]:
#! conda install -c plotly plotly

In [ ]:
death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
country_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')

In [ ]:
death_df.head()

In [ ]:
confirmed_df.tail(10)

In [ ]:
recovered_df.head()

In [ ]:
country_df.head()

In [ ]:
#from pandas import DataFrame
#df = DataFrame (columns = ['County_Region', 'Last_update', 'Lat', 'Long_', 'Confirmed'])
#country_df.loc[df['Country_Region'] == 'kenya']

In [ ]:
#datacleaning
death_df.columns = map(str.lower, death_df.columns )
confirmed_df.columns = map(str.lower, confirmed_df.columns)
recovered_df.columns = map(str.lower, recovered_df.columns)
country_df.columns = map(str.lower, country_df.columns)
country_df.columns 

In [ ]:
death_df = death_df.rename(columns = {"province/state": "state", "country/region": "country"})
confirmed_df = confirmed_df.rename(columns = {"province/state": "state", "country/region": "country"}) 
recovered_df = recovered_df.rename(columns = {"province/state": "state", "country/region": "country"})
country_df = country_df.rename(columns = {"country_region": "country"})

In [ ]:
sorted_country_df = country_df.sort_values("confirmed", ascending=False).head()

In [ ]:
sorted_country_df

In [ ]:
def highlight_col(x):
    r = 'background-color: green'
    p = 'background-color: red'
    g = 'background-color: yellow'
    temp_df = pd.DataFrame('', index=x.index, columns = x.columns)
    temp_df.iloc[:,4] = p
    temp_df.iloc[:,5] = r
    temp_df.iloc[:,6] = g
    return temp_df


def show_latest_cases(n):
    n = int(n)
    return country_df.sort_values('confirmed', ascending= False).head(n).style.apply(highlight_col, axis=None)

interact(show_latest_cases, n='10')

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none'
#widgets.VBox([fig], layout=ipywLayout)

sorted_country_df = country_df.sort_values('confirmed', ascending= False)

In [ ]:
#plot a visual graph
def bubble_chart(n):
    fig = px.scatter(sorted_country_df.head(n), x="country", y="confirmed", size="confirmed", color="country",
               hover_name="country", size_max=80)
    fig.update_layout(
    title=str(n) +" Worst hit countries",#style={"style=center"},
    xaxis_title="Countries",
    yaxis_title="Confirmed Cases",
    #bgcolor="white",
    paper_bgcolor="skyblue",
    width = 900,
        
    )
    
    #fig.show();
    return fig

interact(bubble_chart, n=12)
ipywLayout = widgets.Layout(border='solid 2px green')
style={"style=center"}
#ipywLayout.display='none'



In [ ]:
def plot_cases_for_country(country):
    labels = ['confirmed', 'deaths']
    colors = ['blue', 'red']
    mode_size = [6,8]
    line_size = [4,5]
    
    df_list = (confirmed_df, death_df)
    
    fig = go.Figure()
    
    for i, df in enumerate(df_list):
        if country == 'World' or country == 'world':
            x_data = np.array(list(df.iloc[:, 5:].columns))
            y_data = np.sum(np.asarray(df.iloc[:, 5:]), axis = 0)
        else:    
            x_data = np.array(list(df.iloc[:, 5:].columns))
            y_data = np.sum(np.asarray(df[df['country'] == country].iloc[:,5:]),axis = 0)
            
        fig.add_trace(go.Scatter(x=x_data, y=y_data, mode='lines+markers', name=labels[i], line=dict(color=colors[i], width=line_size[i]),
        connectgaps=True,
        text = "Total " + str(labels[i]) +": "+ str(y_data[-1])));
        
        
    fig.update_layout(
        title="COVID 19 cases of " + country,
        xaxis_title='Date',
        yaxis_title='No. of Confirmed Cases',
        margin=dict(l=20, r=20, t=40, b=20),
        paper_bgcolor="skyblue",
        #gcolor = "grey",
        width = 800,);
    
    fig.update_yaxes(type="linear")
    fig.show()
        
#plot_cases_for_country('Kenya')
interact(plot_cases_for_country, country = 'World');
        

In [ ]:
world_map = folium.Map(location = [11,0], tiles = "cartodbpositron", zoom_start = 2, max_zoom = 8, min_zoom = 1)

for i in range(len(confirmed_df)):
    folium.Circle(location = [confirmed_df.iloc[i]['lat'], confirmed_df.iloc[i]['long']], fill = True, 
    radius = (int((np.log(confirmed_df.iloc[i,-1]+1.00001))) + 0.2)*50000,
    fill_color = 'red',
    color = 'blue',
    
    ).add_to(world_map)
    
world_map